<a href="https://colab.research.google.com/github/Niel-77/Geminiagent/blob/main/Multi_Agent_team_for_MechE_problems_using_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Creating two agent team using Gemini
This is the notebook for creating conversable agent using gemini api. In this notebook we will create a conversable agent to solve a simple mechanical engineering problem. We will be usingpython package, FEniCS65 as the simulation environment for solving elasticity problems using FEM.


## Creating Gemini API key
We	can obtain Gemini API key from https://aistudio.google.com/. Gemini 1.0, Gemini 1.5 Pro and Gemini 1.5 Flash can be used for free of charge but with limits set to each of them for Requests per minutes (RPM), tokens per minute (TPM) and requests per day (RPD).

## Preparing FeniCS for Colab
We will install the fenics package using the github link if it not installed already.

In [1]:
# try:
#     import dolfin
# except ImportError:
#     !wget "https://fem-on-colab.github.io/releases/fenics-install-real.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
#     import dolfin

## Prepare autogen for collab

In [2]:
try:
    import chromadb
except ImportError:
    !pip install chromadb==0.4.15
    import chromadb

try:
    import tiktoken
except ImportError:
    !pip install tiktoken==0.5.1
    import tiktoken

try:
    import pypdf
except ImportError:
    !pip install pypdf==3.17.0
    import pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.8/479.8 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 18

## Install Autogen with gemini features

In [3]:
pip install pyautogen[gemini]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.6/284.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 29.6 MB/s eta 0:00:00


## Install the necessary dependencies

In [4]:

# Install pyautogen with the required extras
!pip install pyautogen[gemini,retrievechat,lmm] --use-deprecated=legacy-resolver


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 36.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_

## Importing the necessary functions
(All the imported functions might not have been used)

In [5]:
import os
from typing import Any, Callable, Dict, List, Optional, Tuple, Type, Union

import chromadb
from PIL import Image
from termcolor import colored

import autogen
from autogen import Agent, AssistantAgent, ConversableAgent, UserProxyAgent
from autogen.agentchat.contrib.img_utils import _to_pil, get_image_data
from autogen.agentchat.contrib.multimodal_conversable_agent import MultimodalConversableAgent
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from autogen.code_utils import DEFAULT_MODEL, UNKNOWN, content_str, execute_code, extract_code, infer_lang

## Creating configuration for gemini and setting API endpoint
The similar configuration model can be used for all LLM models. For using gpt along with gemini, for example, we can add another list of called *config_list_gpt* with the model name and api key. This might not be the best way to create configuration when using multiple agent team but this will help us to avoid OAI_CONFIG_LIST which I found to be a little bit confusing to use with colab.

In [6]:
from google.colab import userdata
geminiapi=userdata.get('GEMINIAPI')
config_list_gemini = [
     {
        "model": "gemini-pro",
        "api_key": geminiapi,
        "api_type": "google"
    }
]


In [7]:
gemini_config = {
    "seed": 25,  # change the seed for different trials
    "temperature": 0,
    "config_list": config_list_gemini,
    "request_timeout": 600,
    "retry_wait_time": 120,
}

## Creating assistants
Here we create assistants with suitable system message.

In [8]:
user_proxy = autogen.UserProxyAgent(
   name="Admin",
   system_message="A human admin. Interact with the planner to discuss the plan. Plan execution needs to be approved by this admin.",
   code_execution_config=False,
)

# engineer = autogen.AssistantAgent(
#     name="Engineer",
#     llm_config=gemini_config,
#     system_message='''Engineer. You follow an approved plan. You write python/shell code to solve tasks. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor.
# Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor.
# If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
# ''',
# )
engineer = AssistantAgent(
    name="Engineer",
    llm_config=gemini_config,
    system_message='''Engineer. You follow an approved plan. You write python/shell code to solve tasks. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor.
Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
In the code you write, always add a part to report the solution on the boundaries and store it in a seperated file for the Scientist to check.
''',
)

# scientist = autogen.AssistantAgent(
#     name="Scientist",
#     llm_config=gemini_config,
#     system_message="""Scientist. You follow an approved plan. You are able to categorize papers after seeing their abstracts printed. You don't write code."""
# )
#
scientist = autogen.AssistantAgent(
    name="Scientist",
    llm_config=gemini_config,
    system_message="""Scientist. You follow an approved plan. You are able to formulate the mechanics problem with clear boundary condition and constitutive law of materails. You don't write code.
    You explicit check the boundary results from the Engineer to see whether it agrees with the input boundary condition.
    When you excute the code, always save a copy for review."""
)

planner = autogen.AssistantAgent(
    name="Planner",
    system_message='''Planner. Suggest a plan. Revise the plan based on feedback from admin and critic, until admin approval.
The plan may involve an engineer who can write code and a scientist who doesn't write code.
Explain the plan first. Be clear which step is performed by an engineer, and which step is performed by a scientist.
''',
    llm_config=gemini_config,
)

executor = autogen.UserProxyAgent(
    name="Executor",
    system_message="Executor. Save and execute the code written by the engineer and report and save the result.",
    human_input_mode="NEVER",
    code_execution_config={"last_n_messages": 3, "work_dir": "Mechanics"},
)

critic = autogen.AssistantAgent(
    name="Critic",
    system_message="Critic. Double check plan, claims, code from other agents, results on the boundary conditions and provide feedback. Check whether the plan includes adding verifiable info such as source URL.",
    llm_config=gemini_config,
)

groupchat = autogen.GroupChat(agents=[user_proxy, planner, engineer, scientist, executor, critic], messages=[], max_round=50)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gemini_config)

## Provide question to the engineer
The chat manager receives the task description from the user and broadcast it to the relevant assistants.

In [12]:
user_proxy.initiate_chat(
    manager,
    message="""A force of 12KN is acting vertically at free end in a cantilever beam of length 4m.\
    Take E=200 GPa and I= 0.36m^-4. Use two linear elements and find nodal displacements at three nodes in the beam.""",
)

Admin (to chat_manager):

A force of 12KN is acting vertically at free end in a cantilever beam of length 4m.    Take E=200 GPa and I= 0.36m^-4. Use two linear elements and find nodal displacements at three nodes in the beam.

--------------------------------------------------------------------------------

Next speaker: Planner

Planner (to chat_manager):

**Plan:**

**Step 1: Engineer**
* Create a finite element model of the cantilever beam using two linear elements.
* Define the material properties (E = 200 GPa) and geometric properties (I = 0.36 m^-4).
* Apply the force of 12 kN at the free end of the beam.

**Step 2: Scientist**
* Solve the finite element equations to determine the nodal displacements at the three nodes in the beam.

**Step 3: Engineer**
* Review the results and ensure that they are reasonable.
* If necessary, refine the finite element model or adjust the material properties to improve the accuracy of the results.

**Step 4: Scientist**
* Interpret the results and

RuntimeError: Google GenAI exception occurred while calling Gemini API: index: 0
finish_reason: SAFETY
safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: MEDIUM
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: NEGLIGIBLE
}
